# Multiperiod cash planning

$$ \text{Minimize   } 0.015\sum_{i=1}^5 B_{i}-0.005\sum_{i=1}^5 M_{i}+0.06s+\sum_{i=1}^5 [(1/0.98-1)(D_{i}-A_{i})]\\
s.t. \\
\text{End-of-month cash balance}: M_i \ge 0.25,i=1,2,...,6\\
\text{Month 1}: 0.4+1.5+0.005*0.4+S+B_1=A1+0.01S++M_1\\
\text{Month 2}: M_1+1.0+0.005M1+B_2 = A2 +0.01S +1.015B_1 +(1.8-A1)/0.98 +M_2 \\
\text{Month 3}: M_2+1.4+0.005M2+B_3 = A3 +0.01S +1.015B_2 +(1.6-A2)/0.98 +M_3 \\
\text{Month 4}: M_3+2.3+0.005M3+B_4 = A4 +0.01S +1.015B_3 +(2.2-A3)/0.98 +M_4 \\
\text{Month 5}: M_4+2.0+0.005M4+B_5 = A5 +0.01S +1.015B_4 +(1.2-A4)/0.98 +M_5 \\
\text{Month 6}: M_5+1.0+0.005M5+B_6 = A6 +1.01S +1.015B_5 +(0.8-A5)/0.98 +M_6\\
$$

## Step 1: Import PuLP modeler functions

In [63]:
from pulp import *
import pandas as pd

## Step 2.1: Use LpProblem, LpVariable to define LP problem and decision variables. 

In [64]:
#define LP problem
probA=LpProblem("Cash",LpMinimize)

#specify parameters
month = list(range(1,7))
receivable={
1:1.5,
2:1,
3:1.4,
4:2.3,
5:2,
6:1     
}
balance={
1:1.8,
2:1.6,
3:2.2,
4:1.2,
5:0.8,
6:1.2   
}

delay_rate=1/0.98

min_balance=0.25
init_balance=0.4


# define decision variables
delay_payment=LpVariable.dicts("d",month,lowBound=0, cat='Continuous')
borrow_against=LpVariable.dicts("b",month,lowBound=0, cat='Continuous')
short_term=LpVariable("s",lowBound=0, cat='Continuous')
m_vars=LpVariable.dicts("m",month,lowBound=min_balance,cat='Continuous')
print(delay_payment)
print(borrow_against)
print(short_term)
print(m_vars)

{1: d_1, 2: d_2, 3: d_3, 4: d_4, 5: d_5, 6: d_6}
{1: b_1, 2: b_2, 3: b_3, 4: b_4, 5: b_5, 6: b_6}
s
{1: m_1, 2: m_2, 3: m_3, 4: m_4, 5: m_5, 6: m_6}


## Step 2.2:  Use +=, lpSum() to formulate the linear program model

In [65]:
#objective function
b_subtotal=[0]*7
for i in range(2,7):
    b_subtotal[i]=0.015*borrow_against[i-1]-0.005*m_vars[i-1]+0.01*short_term+(1/0.98-1)*\
    delay_payment[i-1]

probA += lpSum([b_subtotal[i] for i in range(2,7)])-0.005*0.4+0.01*short_term

#constraints

# inflow-outflow balance constraints

# month 1 inflow-outflow balance
probA += init_balance+receivable[1]+0.005*init_balance+short_term+borrow_against[1]\
== balance[1]-delay_payment[1]+m_vars[1]+0.01*short_term
# month 2 to month 6 inflow-outflow balance
for i in range(2,6):
    probA += m_vars[i-1]+receivable[i]+0.005*m_vars[i-1]+borrow_against[i]\
    == balance[i]-delay_payment[i]+0.01*short_term+1.015*borrow_against[i-1]+m_vars[i]\
    +delay_payment[i-1]*delay_rate
# month 6 inflow-outflow balance    
probA += m_vars[5]+receivable[6]+0.005*m_vars[5]\
== balance[6]+1.01*short_term+1.015*borrow_against[5]+m_vars[6]\
+delay_payment[5]*delay_rate
# less than 75%
for i in range(1,7):    
    probA += borrow_against[i] <= receivable[i]*0.75
delay_payment[6]=0
borrow_against[6] = 0
probA


Cash:
MINIMIZE
0.015*b_1 + 0.015*b_2 + 0.015*b_3 + 0.015*b_4 + 0.015*b_5 + 0.020408163265306145*d_1 + 0.020408163265306145*d_2 + 0.020408163265306145*d_3 + 0.020408163265306145*d_4 + 0.020408163265306145*d_5 + -0.005*m_1 + -0.005*m_2 + -0.005*m_3 + -0.005*m_4 + -0.005*m_5 + 0.060000000000000005*s + -0.002
SUBJECT TO
_C1: b_1 + d_1 - m_1 + 0.99 s = -0.102

_C2: - 1.015 b_1 + b_2 - 1.02040816327 d_1 + d_2 + 1.005 m_1 - m_2 - 0.01 s
 = 0.6

_C3: - 1.015 b_2 + b_3 - 1.02040816327 d_2 + d_3 + 1.005 m_2 - m_3 - 0.01 s
 = 0.8

_C4: - 1.015 b_3 + b_4 - 1.02040816327 d_3 + d_4 + 1.005 m_3 - m_4 - 0.01 s
 = -1.1

_C5: - 1.015 b_4 + b_5 - 1.02040816327 d_4 + d_5 + 1.005 m_4 - m_5 - 0.01 s
 = -1.2

_C6: - 1.015 b_5 - 1.02040816327 d_5 + 1.005 m_5 - m_6 - 1.01 s = 0.2

_C7: b_1 <= 1.125

_C8: b_2 <= 0.75

_C9: b_3 <= 1.05

_C10: b_4 <= 1.725

_C11: b_5 <= 1.5

_C12: b_6 <= 0.75

VARIABLES
b_1 Continuous
b_2 Continuous
b_3 Continuous
b_4 Continuous
b_5 Continuous
b_6 Continuous
d_1 Continuous
d_2 Co

## Step 4: Run solver

- use name.solve(solver=None), where name is the LP problem variable defined by LpProblem
- Solve the given Lp problem. 
- This function changes the problem to make it suitable for solving then calls the solver.actualSolve method to find the solution. 
- solver – Optional: the specific solver to be used, defaults to the default solver.

In [66]:
probA.writeLP("Planning_cash.lp")
probA.solve()
print("Status:",LpStatus[probA.status])

Status: Optimal


## Step 5: Print the optiomal solution

In [67]:
for v in probA.variables():
    print(v.name, "=", v.varValue,"\tReduced Cost =", v.dj)
    
print("Total cost=", value(probA.objective))

# sensitivity analysis information
print("\nSensitivity Analysis\nConstraint\t\t\t\t\tShadow Price\tSlack")
for name, c in list(probA.constraints.items()):
    print(name, ":", c, "\t", c.pi, "\t\t", c.slack)

b_1 = 0.0 	Reduced Cost = 0.010510671
b_2 = 0.60024495 	Reduced Cost = -6.9388939e-18
b_3 = 1.05 	Reduced Cost = 0.0
b_4 = 0.33282513 	Reduced Cost = 3.469447e-18
b_5 = 0.0 	Reduced Cost = 0.01
b_6 = 0.0 	Reduced Cost = 0.0
d_1 = 0.0 	Reduced Cost = 0.016224429
d_2 = 0.0 	Reduced Cost = 0.0056293185
d_3 = 0.35949357 	Reduced Cost = -3.469447e-18
d_4 = 0.0 	Reduced Cost = 0.0054352041
d_5 = 0.0 	Reduced Cost = 0.015408163
m_1 = 0.25 	Reduced Cost = 5.4391549e-05
m_2 = 0.25 	Reduced Cost = 0.010408929
m_3 = 0.25 	Reduced Cost = 0.015717482
m_4 = 0.25 	Reduced Cost = 0.01005
m_5 = 1.1119375 	Reduced Cost = 0.0
m_6 = 0.76650734 	Reduced Cost = 0.0
s = 0.14949495 	Reduced Cost = 8.3280279e-18
Total cost= 0.03349266416938776

Sensitivity Analysis
Constraint					Shadow Price	Slack
_C1 : b_1 + d_1 - m_1 + 0.99*s = -0.10199999999999987 	 0.061843173 		 1.249000902703301e-16
_C2 : -1.015*b_1 + b_2 - 1.0204081632653061*d_1 + d_2 + 1.005*m_1 - m_2 - 0.01*s = 0.6000000000000001 	 0.05650625 		 1.11

## Step 6: Format Output

## Format Shadow Price Output

In [68]:
output=[]

borrowagainst = [borrow_against[i].varValue for i in range(1,6)]
borrowagainst.extend([0])

delaypayment = [delay_payment[i].varValue for i in range(1,6)]
delaypayment.extend([0])

shortterm = [short_term.varValue]
shortterm.extend([0]*5)

cashonhand = [m_vars[i].varValue for i in range(1,7)]

output.append(borrowagainst)
output.append(delaypayment)
output.append(shortterm)
output.append(cashonhand)
print(output)  

col_names=['Month 1', 'Month 2', 'Month 3', 'Month 4', 'Month 5', 'Month 6']
row_names=['borrowagainst', 'delaypayment', 'shortterm', 'cashonhand']
output_df = pd.DataFrame(output,index=row_names, columns=col_names)

print("Total cost=", value(probA.objective))
output_df

[[0.0, 0.60024495, 1.05, 0.33282513, 0.0, 0], [0.0, 0.0, 0.35949357, 0.0, 0.0, 0], [0.14949495, 0, 0, 0, 0, 0], [0.25, 0.25, 0.25, 0.25, 1.1119375, 0.76650734]]
Total cost= 0.03349266416938776


,Month 1,Month 2,Month 3,Month 4,Month 5,Month 6
borrowagainst,0.000000,0.600245,1.050000,0.332825,0.000000,0.000000
delaypayment,0.000000,0.000000,0.359494,0.000000,0.000000,0.000000
shortterm,0.149495,0.000000,0.000000,0.000000,0.000000,0.000000
cashonhand,0.250000,0.250000,0.250000,0.250000,1.111938,0.766507
